## The property class

The property class can be instantiated in different ways:

```python
x = property(fget = get_x, fset = set_x)
```

The class defines methods (getter, setter, deleter) that can take a callable as an argument and returns the instance with appropriate method

Could create it this way instead:

```python
x = property()
x = x.getter(get_x)
x = x.setter(set_x)
```

or 

``` python
x = property(get_x)
x = x.setter
```

```python
def MyClass:
    def __init__(self, language):
        self._language = language
        
    def language(self):
        return self._language
    
    language = property(language)
```
We now have a __property__ language with a getter method defined. remind you of decorator?


instead we can write:

```python
def MyClass:
    def __init__(self, language):
        self._language = language
        
    def language(self):
        return self._language 
    
    
Next we may want to to define a settter method as well

```python
def MyClass:
    def __init__(self, language):
        self._language = language

    @property
    def language(self):
        return self._language 
        # at this point language is now a property instance
        
    def set_language(self, value):
        self._language = value
        # this is a setter method which we need to assign to the language property
        
    language = language.setter(set_language)
```

But again, we can rewrite this using the @ decorator syntax

```python
@language.setter
def language(self, value):
    self._language = value
```


To summarize, we can use decorators to create property type objects as well

```python
def MyClass:
    def __init__(self, language):
        self._language = language

    @property
    def language(self):  # function name defines the property instance name (symbol)
        return self._language

    # language is now a property instance (an object)
    @language.setter   # we use the setter method of the language property object
    def language(self, value) # important to use the same name, otherwise we end up with a new symbol for our property!
        self._language = value
```


### Property Decorators

As I explain the `property` callable actually returns itself:

Think back to how decorators work:

In [3]:
def my_decorator(fn):
    print('decorating function')
    def inner(*args, **kwargs):
        print('running decorated function')
        return fn(*args, **kwargs)
    return inner

In [4]:
def undecorated_function(a, b):
    print('running original function')
    return a + b

Now we can decorate our undecorated function this way:

In [5]:
decorated_func = my_decorator(undecorated_function)

decorating function


And we can call the decorated function:

In [6]:
decorated_func(10, 20)

running decorated function
running original function


30

Now instead of giving the decorate function a new symbol, we could have just re-used the same symbol:

In [7]:
def my_func(a, b):
    print('running original function')
    return a + b

my_func = my_decorator(my_func)

decorating function


In [8]:
my_func(10, 20)

running decorated function
running original function


30

And of course this is exactly what the decorator `@` syntax does:

In [9]:
@my_decorator
def my_func(a, b):
    print('running original function')
    return a + b

decorating function


In [10]:
my_func(10, 20)

running decorated function
running original function


30

Ok, now that we've refreshed our memory on decorators, we should be ready to look at the `property` callable.

The `property` callable creates a property object, **and returns it**.

In other words, we could create our property this way, as usual:

In [11]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def name(self):
        return self._name
    
    name = property(name)

In [12]:
p = Person('Alex')

p.name

'Alex'

But you'll notice that line: `name = property(name)` - that's exactly what the decorator syntax does for us!

So instead we can write:

In [13]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

In [14]:
p = Person('Guido')
p.name

'Guido'

Well, the `property` object has some properties, like `setter` that will basically accept a reference to the setter method, and return itself also.

In [24]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # what's the property name now? --> name
    # so name has a setter callable
    @name.setter
    def name(self, value):
        self._name = value

In [26]:
p = Person('Alex')

In [27]:
p.name

'Alex'

In [28]:
p.name = 'Guido'
p.name

'Guido'

Technically, the property callable has both a getter and setter method - so we can create the setter first, then "add in" the getter. But since the first argument to `property` is the getter, we have to work a bit more to do it:

In [37]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value

By the way, we now have a property that can be set, but not read back!

In [38]:
p = Person('Alex')

In [39]:
p.__dict__

{'_name': 'Alex'}

In [40]:
p.name = 'Raymond'

In [41]:
p.__dict__

{'_name': 'Raymond'}

In [42]:
try:
    p.name
except AttributeError as ex:
    print(ex)

unreadable attribute


So, if you ever need an attribute that is "write-only" - you can do it. Maybe the data is sensitive, and you want to set it, but not show back to users... But the data is never truly private, so at best you're obfuscating the data - so in my experience I've never had to do something like that. Just wanted you to see this in case the need ever came up.

But let's finish this up and make the property read/write:

In [43]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value
        
    @name.getter
    def name(self):
        return self._name

In [44]:
p = Person('Alex')

In [45]:
p.name

'Alex'

In [46]:
p.name = 'Raymond'

In [47]:
p.name

'Raymond'

The deleter works the same way, and we'll come back to it soon.

Lastly you'll recall that we could set up a docstring when using the `property` callable.

The standard technique is to simply define the docstring in the getter function:

In [48]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        """The Person's name."""
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

In [49]:
help(Person.name)

Help on property:

    The Person's name.



In [50]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  name
 |      The Person's name.



What happens if we set it in the setter instead?

In [51]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        """The Person's name."""
        self._name = value

In [52]:
help(Person.name)

Help on property:




In [53]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  name



As you can see, the property docstring is only set on the getter. So how to set a docstring with a write-only property? We can do that when we create the initial property:

In [54]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property(doc='Write-only name property.')
    
    @name.setter
    def name(self, value):
        self._name = value

In [55]:
help(Person.name)

Help on property:

    Write-only name property.

